# 3INFO 2024-2025 - Feuille de calculs pour les examens

## Data model and Helpers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
# from matplotlib.backends.backend_pdf import PdfPages
import pandas.io.formats.style as pdstyle
import os
from io import StringIO
from os import listdir
from os.path import isfile, join
from typing import TypeVar
from collections.abc import Callable, Iterable
from functools import reduce
from __future__ import annotations # type: ignore

# The code of this cell works for any year

# creating folders

if not os.path.exists(os.path.join('graphics','stds')):
    os.makedirs(os.path.join('graphics','stds'))

pd.set_option('display.max_rows', 300)

# Existing marks
# override the marking when students keeps their marking from the past year
class ExtraMark:
    def __init__(self, name: str, marks: list[tuple[int, float, bool]]) -> None:
         # The name of the exam (the official name in Pegase of the exam)
        self.name = name
        # the tuple: the student ID, the mark, whether the student is 'redoublant/aménagement'
        self.marks = marks

    def __str__(self) -> str:
        return '''%s %s''' % (self.name, self.marks)

    def __repr__(self) -> str:
        return str(self)


def getMark(marks: pd.DataFrame, code: str, nom: str) -> float:
    try:
        return float(marks[marks["LIBELLE_CONTROLE"] == code]['NOTE'].values[0])
    except:
        # print("Pas de note en %s pour %s" % (code, nom))
        return np.nan

def getCredits(mark: float, credits: float, mark_eu: float = 0) -> float:
    return credits if (mark >= 10) | (mark_eu >= 10) else 0

def isLower10(value: pd.Series[float]) -> pd.Series[bool]:
    return ~pd.isna(value) & (value < 10)

def highlight_everyother(s: pd.Series[float]) -> list[str]:
    return ['background-color: lightgray; color:black;' if x % 2==1 else '' for x in range(len(s))]

def compute_module_mark(tuples: list[tuple[float, float]]) -> float:
    a = sum(map(lambda tuple: 0.0 if pd.isna(tuple[0]) or tuple[0] == 30 else tuple[0] * tuple[1], tuples))
    b = sum(map(lambda tuple: 0.0 if pd.isna(tuple[0]) or tuple[0] == 30 else tuple[1], tuples))
    return float('nan') if b == 0.0 else a / b

def isFisp(id: int, specificities: pd.DataFrame) -> bool:
    return bool((specificities[specificities["ID"] == id]["Carac"] == "FISP").any())

# Adds metadata to the dataframe of a semester
def insert_metadata(df: pd.DataFrame, header: list[str]) -> pd.DataFrame:
    means: pd.Series[float] = df[header].mean(axis=0)
    medians: pd.Series[float] = df[header].median(axis=0)
    mins: pd.Series[float] = df[header].min(axis=0)
    maxs: pd.Series[float] = df[header].max(axis=0)
    stds: pd.Series[float] = df[header].std(axis=0)

    res: pd.DataFrame = pd.concat([means.to_frame().T, medians.to_frame().T, mins.to_frame().T, maxs.to_frame().T, stds.to_frame().T], ignore_index=True)
    res.reindex(["Moyenne", "Médiane", "Min", "Max", "Std dev"], axis="columns")
    return res

# These methods permit the highlightning of the columns of each UE
style_column_ue_ko = ["background-color: red; color: black;font-weight: bold;", "background-color: orange; color: black;font-weight: bold;"]
style_column_ue_ok = "background-color: green; color: black;font-weight: bold;"

# To be used while styling the table of a semester. Highlights a specific column
def highlighterUE(col: pd.Series[float], /, predicate: pd.Series[bool]) -> list[np.bool_]:
    res: list[np.bool_] = np.select([col<10, predicate], style_column_ue_ko, style_column_ue_ok).tolist() # force casting to np.bool_
    return res

T = TypeVar('T')
S = TypeVar('S')
def flat_map(f: Callable[[Iterable[T]], Iterable[S]], xs: list[list[T]]) -> list[S]:
    ys: list[S] = []
    x: list[T]
    for x in xs:
        ys.extend(f(x))
    return ys

class Exam:
    def __init__(self: Exam, pegase_name: str, short_name: str | None = None, coeff: float = 1) -> None:
        self.pegase_name = pegase_name
        self.coeff = coeff
        self.short_name = short_name

    def get_name_display(self: Exam) -> str:
        return self.pegase_name if self.short_name is None else self.short_name

    def get_std_mark(self: Exam, df: pd.DataFrame, std_name: str) -> float:
        return getMark(df, self.pegase_name, std_name)


class Course:
    def __init__(self: Course, name: str, ects: float, exams: list[Exam], ects_for_special: float = np.nan, no_course_for_special = False) -> None:
        self.name = name
        self.ects = ects
        self.__exams = exams
        self.ects_for_special = ects_for_special
        self.no_course_for_special = no_course_for_special

    def get_exams(self: Course) -> list[Exam]:
        return self.__exams

    def get_header_table(self: Course) -> list[str]:
        return [self.name] if len(self.get_exams()) == 1 else list(map(lambda x: x.get_name_display(), self.get_exams())) + [self.name]

    def get_subexam_names(self: Course) -> list[str]:
        return [] if len(self.get_exams()) == 1 else list(map(lambda x: x.get_name_display(), self.get_exams()))

    def is_validating_course(self: Course, df: pd.DataFrame) -> pd.Series[bool]:
        return isLower10(df[self.name])

    def get_std_mark(self: Course, df: pd.DataFrame, std_name: str) -> float:
        return self.get_exams()[0].get_std_mark(df, std_name) if len(self.get_exams()) == 1 else \
            compute_module_mark(list(map(lambda x : (x.get_std_mark(df, std_name), x.coeff), self.get_exams())))

    def get_std_ECTS_strict(self: Course, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        fisp = isFisp(std_id, special_students)
        credits = self.get_ECTS(fisp)
        return credits if fisp and self.no_course_for_special else \
              getCredits(self.get_std_mark(df, std_name), credits)

    def get_ECTS(self: Course, fisp: bool) -> float:
        return self.ects if pd.isna(self.ects_for_special) or not fisp else self.ects_for_special

    def get_std_marks(self: Course, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> list[float]:
        return [self.get_std_mark(df, std_name)] if len(self.get_exams()) == 1 else \
            list(map(lambda x: x.get_std_mark(df, std_name), self.get_exams())) + [self.get_std_mark(df, std_name)]


class CourseOr(Course):
    def __init__(self: CourseOr, name: str, ects: float, sub_courses: list[Course]) -> None:
        super().__init__(name, ects, [])
        self.sub_courses = sub_courses

    def get_exams(self: CourseOr) -> list[Exam]:
        return flat_map(lambda x : x, list(map(lambda x: x.get_exams(), self.sub_courses)))

class UE:
    def get_header_table(self: UE) -> list[str]:
        return flat_map(lambda x: x, list(map(lambda x: x.get_header_table(), self.courses))) + [self.name]

    def __init__(self, name: str, courses: list[Course], consider_in_global_mean: bool = True) -> None:
        self.name = name
        self.courses = courses
        self.consider_in_global_mean = consider_in_global_mean

    def get_ects(self) -> float:
        return sum(map(lambda course: course.ects, self.courses))

    def get_token_courses(self: UE) -> list[str]:
        return list(map(lambda course: course.name, self.courses))

    def get_subexam_names(self: UE) -> list[str]:
        return flat_map(lambda x : x, list(map(lambda ue: ue.get_subexam_names(), self.courses)))

    def is_validating_UE(self: UE, df: pd.DataFrame) -> pd.Series[bool]:
        return reduce(lambda x, y: x|y, list(map(lambda ue: ue.is_validating_course(df), self.courses)))

    def get_std_mark(self: UE, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        return compute_module_mark(list(map(lambda x : (x.get_std_mark(df, std_name), x.ects), self.courses)))

    def get_std_marks(self: UE, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> list[float]:
        return flat_map(lambda x: x, list(map(lambda x: x.get_std_marks(df, std_name, std_id, special_students), self.courses))) \
            + [self.get_std_mark(df, std_name, std_id, special_students)]

    def get_std_ECTS_strict(self: UE, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        return reduce(lambda x, y: x+y, list(map(lambda x: x.get_std_ECTS_strict(df, std_name, std_id, special_students), self.courses)))

    def get_std_ECTS_compensation(self: UE, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        mark_UE = self.get_std_mark(df, std_name, std_id, special_students)
        if pd.isna(mark_UE):
            return 0.0
        if mark_UE < 10.0:
            return self.get_std_ECTS_strict(df, std_name, std_id, special_students)
        return reduce(lambda x, y: x + y, list(map(lambda x: x.get_ECTS(isFisp(std_id, special_students)), self.courses)))


class Semester:
    def __init__(self: Semester, name: str, folder_exams: str, ues: list[UE]) -> None:
        self.name = name
        self.ues = ues
        self.folder_exams = folder_exams
        self.expected_ects = 30.0
        self.critical_ects_level = 10.0
        self.passable_ects_level = 25.0

    def get_header_table(self: Semester) -> list[str]:
        return [self.get_token_Rank(), self.get_token_Avg(), self.get_token_ECTS(), self.get_token_ECTS_compensation()] \
            + flat_map(lambda x: x, list(map(lambda x: x.get_header_table(), self.ues)))

    def get_UE_names(self: Semester) -> list[str]:
        return list(map(lambda ue: ue.name, self.ues))

    def get_UE_selectors(self: Semester, gap: int) -> str:
        header = self.get_header_table()
        return ", ".join(list(map(lambda ue: ".col%s" % (header.index(ue.name) + gap), self.ues)))

    def get_course_names(self: Semester) -> list[str]:
        return flat_map(lambda x : x, list(map(lambda ue: ue.get_token_courses(), self.ues)))

    def get_subexam_names(self: Semester) -> list[str]:
        return flat_map(lambda x : x, list(map(lambda ue: ue.get_subexam_names(), self.ues)))

    def get_ects(self: Semester) -> float:
        return sum(map(lambda ue: ue.get_ects(), self.ues))

    def check_ects(self: Semester) -> None:
        assert self.get_ects() == 30.0, "Incorrect number of ECTS points for the semester %s" %(self.name)

    def get_token_ECTS(self: Semester) -> str:
        return 'ECTS %s'%(self.name)

    def get_token_ECTS_compensation(self: Semester) -> str:
        return 'ECTS+ %s'%(self.name)

    def get_token_Avg(self: Semester) -> str:
        return 'Avg %s' %(self.name)

    def get_token_Rank(self: Semester) -> str:
        return 'Rank'

    def get_UEs_to_consider(self: Semester) -> list[UE]:
        return list(filter(lambda x: x.consider_in_global_mean, self.ues))

    def get_std_mean(self: Semester, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        return compute_module_mark(list(map(lambda x : (x.get_std_mark(df, std_name, std_id, special_students),
                                                        x.get_ects()), self.get_UEs_to_consider())))

    def get_std_ECTS_strict(self: Semester, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        return reduce(lambda x, y : x + y, list(map(lambda x: x.get_std_ECTS_strict(df, std_name, std_id, special_students), self.ues)))

    def get_std_ECTS_compensation(self: Semester, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> float:
        return reduce(lambda x, y : x + y, list(map(lambda x: x.get_std_ECTS_compensation(\
            df, std_name, std_id, special_students), self.ues))) + (3.0 if isFisp(std_id, special_students) else 0.0)

    def get_std_marks(self: Semester, df: pd.DataFrame, std_name: str, std_id: int, special_students: pd.DataFrame) -> list[float]:
        return flat_map(lambda x: x, list(map(lambda x: x.get_std_marks(df, std_name, std_id, special_students), self.ues)))



## Year specific variables

In [ ]:
year = "2024-2025"
prevYear = "2023-2024"
year_name = "3INFO"
pictures_folder = "../../../photos/"

semester1_fisp_csv_filename = "examens/%s-3INF-S05-FISP.csv" % (year)
semester1_csv_filename = "examens/%s-3INF-S05.csv" % (year)

semester2_fisp_csv_filename = "examens/%s-3INF-S06-FISP.csv" % (year)
semester2_csv_filename = "examens/%s-3INF-S06.csv" % (year)

# The list of students 'redoublant' or 'aménagement'. Used to include their marks of the previous year
stds_with_specificities_ids: list[int] = []

ueMath = UE("UE Maths", [
    Course("PROBA", 1.5, [Exam("PROBA DS 2h")]),
    Course("ADFD", 2.5, [Exam("ADFD  DS 2h")])])
ueArchi = UE("UE Archi", [
    Course("CLP", 3.0, [Exam("CLP DS 2h")]),
    Course("C", 1.5, [Exam("LANGAGE C DS TP 1h", "TP C"), Exam("LANGAGE C DS 1h", "DS C")]),
    Course("HI", 1.5, [Exam("HI DS 2h")])])
ueProg = UE("UE Prog", [
    Course("PL", 2.0, [Exam("PL DS 1h30")]),
    Course("FUS", 2.5, [Exam("FUS DS 2h", "FUS1", 2.0), Exam("LANGAGE DE SCRIPT  DS 1h", "LDS")]),
    Course("PF", 2.0, [Exam("PF Note finale")])])
ueLog = UE("EU Log", [
    Course("ÉP S5", 2.0, [Exam("EP S5 PROJET")]),
    Course("SDD", 3.0, [Exam("SDD DS 2h")]),
    Course("CPOO1", 1.5, [Exam("CPOO1 DS 2H")])])
ueHumaS5 = UE("EU Huma", [
    Course("RISQ", 1.5, [Exam("Gestion du Risque")]),
    Course("ANG S5", 2.0, [Exam("Controle Continu ANGLAIS S5", "ANG CC"), Exam("DS1 ANGLAIS S5", "ANG DS", 2.0)]),
    Course("PSH", 2.5, [Exam("Note finale Problématiques d'Ingéniérie")], 1.5, True),
    Course("EPS S5", 1.0, [Exam("Controle continu EPS S5")])], False)

semester1 = Semester("S5", "examens/S5/", [ueMath, ueArchi, ueProg, ueLog, ueHumaS5])
semester1.check_ects()


ueRes = UE("UE Res", [
    Course("Res", 1.5, [Exam("RESEAUX DS 2H")]),
    Course("BD", 2.0, [Exam("BASES DE DONNEES DS 1h")]),
    Course("Web1", 2.0, [Exam("PROGRAMMATION WEB DS 1H")])])
ueTheo = UE("UE Théo", [
    Course("GA", 3.0, [Exam("GRAPHES ET ALGO DS 2H")]),
    Course("CX", 2.5, [Exam("COMPLEXITE DS 2H")]),
    Course("Pr", 1.5, [Exam("PROPOSITIONS ET PREDICATS DS 2H")]),
    Course("Appr", 2.5, [Exam("APPRENTISSAGE AUTO DS 2H")])])
ueOpt = UE("UE Opt", [
    CourseOr("SD/ SEC", 2.0, [
        Course("Vuln", 0.0, [Exam("VUNLERABILITES DES SYST INFO DS 2H", "Vuln")]),
        Course("SD", 0.0, [Exam("STATISTIQUES DESCRIPTIVES NOTE TP", "SD TP", 1.0), Exam("STATISTIQUES DISTRIBUEES DS 2", "SD DS", 1.0)])]),
    CourseOr("TAL/ PARAL", 2.0, [
        Course("TAL", 0.0, [Exam("TALEO NOTE TP", "Taleo TP", 1.0), Exam("TALEO DS 1H", "Taleo DS", 2.0)]),
        Course("PARAL", 0.0, [Exam("Prog parallèle DS 2H", "PARAL")])])
    ])
ueProj = UE("UE Proj", [
    Course("ÉP S6", 1.5, [Exam("ETUDES PRATIQUES S6 - NOTE FINALE PROJET")]),
    Course("Conf", 0.5, [Exam("CONFERENCES S6 - NOTE FINALE")]),
    # Course("Ouv", 2.0, [ExamOr([Exam("ROBOTIQUE NOTE TP", "ROB"), Exam("IAJ NOTE FINALE", "IAJ"), Exam("PROGRAMMATION MOBILE NOTE TP", "MOB")])])
    CourseOr("Ouv", 2.0, [
        Course("ROB", 0.0, [Exam("ROBOTIQUE NOTE TP", "ROB")]),
        Course("IAJ", 0.0, [Exam("IAJ NOTE FINALE", "IAJ")]),
        Course("MOB", 0.0, [Exam("PROGRAMMATION MOBILE NOTE TP", "MOB")])])
    ])
ueHumaS6 = UE("UE Hum", [
    Course("IND", 1.5, [Exam("Introduction au Numérique Durable")]),
    Course("ANG S6", 2.0, [Exam("Oral ANGLAIS S6", "ANG O"), Exam("DS ANGLAIS S6", "ANG CC")]),
    Course("SIM", 1.5, [Exam("Simulation de Gestion")]),
    Course("EPS S6", 1.0, [Exam("Control Continu EPS S6")]),
    Course("PPI", 1.0, [Exam("Projet Personnel Individualisé S6")], 0, True)], False)

semester2 = Semester("S6", "examens/S6/", [ueRes, ueTheo, ueOpt, ueProj, ueHumaS6])

print(semester2.name)
semester1.check_ects()

other_marks_semester1: list[ExtraMark] = [
    # ExtraMark("Gestion du Risque", [(123, 18.0, True), (456, 18.5, True)]),
]

other_marks_semester2: list[ExtraMark] = [
    # ExtraMark("Oral ANGLAIS S6", [(123, 16, True), (456, 16, True)]),
]


## Chargement des CSV

### CSV promotions

In [ ]:
# To adapt to another year, have to change the name of the CVS files, some messages, possibly the names of the dataframe objects

# Loading the csv of a set of students
def readCSVPromotion(csv_filename: str | StringIO) -> pd.DataFrame:
    return pd.read_csv(csv_filename, sep= ';', header = 0, usecols=["CODE_APPRENANT", "NOM_FAMILLE", "PRENOM"])

def loadEmpty() -> pd.DataFrame:
    return readCSVPromotion(StringIO('CODE_APPRENANT;NOM_FAMILLE;PRENOM'))

df_promo_semester1_fisp: pd.DataFrame = readCSVPromotion(semester1_fisp_csv_filename)
df_promo_semester1: pd.DataFrame = pd.concat([readCSVPromotion(semester1_csv_filename), df_promo_semester1_fisp])
display(df_promo_semester1)
df_marks_semester1: pd.DataFrame = pd.DataFrame()
try:
    df_promo_semester2_fisp: pd.DataFrame = readCSVPromotion(semester2_fisp_csv_filename)
    df_promo_semester2: pd.DataFrame = pd.concat([readCSVPromotion(semester2_csv_filename), df_promo_semester2_fisp])
    display(df_promo_semester2)
    df_marks_semester2: pd.DataFrame = pd.DataFrame()
except:
    print("cannot load the second semester")

# Loading supplementary information about students
specificities: pd.DataFrame = pd.read_csv("examens/specificities.csv", sep= ';', header = 0, usecols=["ID", "Carac"])

## Loading ranking of the previous year: another CSV file that contains the ranking of each student at the end of their previous year.
def readCSVRankingPreviousYear(csv_filename: str | StringIO) -> pd.DataFrame:
    return pd.DataFrame(pd.read_csv(csv_filename, sep= ';', header = 0, usecols=["ID", "RANK"]))

def setupRankingPreviousYear() -> pd.DataFrame:
    try:
        return readCSVRankingPreviousYear("examens/ranking-prev-year.csv")
    except:
        print("Cannot load the ranking of the previous year")
        return pd.DataFrame({"ID": [], "RANK": []})

def get_std_name_from_id(id: int) -> str:
    return df_promo_semester1[df_promo_semester1["CODE_APPRENANT"] == id][["NOM_FAMILLE", "PRENOM"]].iloc[0].str.cat(sep=" ")

ranking_prev_year: pd.DataFrame = setupRankingPreviousYear()


### CSV examens

In [ ]:
# Loading CSV files

def readCSVExam(csv_filename: str | StringIO) -> pd.DataFrame:
    return pd.read_csv(csv_filename, sep= ';', header = 0, usecols=["CODE_OBJET_MAQUETTE", "LIBELLE_OBJET_MAQUETTE", "LIBELLE_CONTROLE", "CODE_APPRENANT", "NOM_FAMILLE", "PRENOM", "NOTE"])

def loadEmpty() -> pd.DataFrame:
    return readCSVExam(StringIO('CODE_OBJET_MAQUETTE;LIBELLE_OBJET_MAQUETTE;LIBELLE_CONTROLE;CODE_APPRENANT;NOM_FAMILLE;PRENOM;NOTE'))

def readExamFile(file: str) -> pd.DataFrame:
    try:
        return readCSVExam(file)
    except Exception as cause:
        print(cause)
        return loadEmpty()

def load_semestre(path: str) -> pd.DataFrame:
    files = [f for f in listdir(path) if isfile(join(path, f))]
    return pd.concat(filter(lambda df: (not df.empty), map(lambda file: readExamFile(join(path, file)), files)), ignore_index=True)

df_exams_semester1: pd.DataFrame = load_semestre(semester1.folder_exams)
# I manually remove the bad tokens 'ABS INJ.' and co that are present in the marks. These tokens break all the computations
# df_exams_semester1.loc[(df_exams_semester1.NOTE == 'ABS. INJ.'), 'NOTE'] = ''

try:
    df_exams_semester2: pd.DataFrame = load_semestre(semester2.folder_exams)
    # df_exams_semester2.loc[(df_exams_semester2.NOTE == 'ABS. INJ.'), 'NOTE'] = ''
except Exception as cause:
    print(cause)
    df_exams_semester2 = loadEmpty()

try:
    df_exams_semester1_prev: pd.DataFrame = load_semestre("../%s/%s" % (prevYear, semester1.folder_exams))
except Exception as cause:
    df_exams_semester1_prev = loadEmpty()
    print(cause)

try:
    df_exams_semester2_prev: pd.DataFrame = load_semestre("../%s/%s" % (prevYear, semester2.folder_exams))
except Exception as cause:
    df_exams_semester2_prev = loadEmpty()
    print(cause)


def compute_new_mark(oldmark: float, newmark: float, repeat_year: bool) -> float:
    # This function considers: 'redoublement', 'aménagement', and 'rattrapage'. The mark from a substitution exam is considered as the mark of the official exam
    # repeat_year => both 'redoublant' ; 'aménagement' (that did not validate the mark during the first try)
    if repeat_year:
        if oldmark >= 10:
            return max(10, newmark)
        return newmark
    # in case of second chance (during the same year) ('Rattrapage')
    return min(10, newmark)

def complete_marks2(stds: list[int], df: pd.DataFrame, dfPrev: pd.DataFrame) -> pd.DataFrame:
    for std in stds:
        std_marks_prev: pd.DataFrame = dfPrev[dfPrev["CODE_APPRENANT"] == std]
        for _, data in std_marks_prev.iterrows():
            mark_prev = float(data["NOTE"])

            if not pd.isna(mark_prev):
                course: str = str(data["LIBELLE_CONTROLE"])
                mark: pd.Series[float] = df.loc[np.logical_and(df["LIBELLE_CONTROLE"] == course, df["CODE_APPRENANT"] == std), "NOTE"].astype("float")

                if len(mark) == 0:
                    print('2:', course, get_std_name_from_id(std), std,  'previous mark (without a new one)', mark_prev)
                    df = pd.concat([df, pd.DataFrame([["","", course, std, "", "", mark_prev]], columns=df.columns)])
                else:
                    curr_mark: float = mark.iloc[0]
                    applied_mark: float = 0.0

                    if pd.isna(curr_mark):
                        applied_mark = mark_prev
                        print('2:', course, get_std_name_from_id(std), std, 'previous mark (without a new one (NaN))', mark_prev)
                    else:
                        # if the mark exists in the exam of the previous year, its means it is a 'redoublement' or a 'aménagement'
                        applied_mark = compute_new_mark(mark_prev, curr_mark, True)
                        print('2:', course, get_std_name_from_id(std), std, 'new mark', curr_mark, 'with a former one', mark_prev, 'considered as redoublement. Final mark', applied_mark)

                    df.loc[np.logical_and(df["LIBELLE_CONTROLE"] == course, df["CODE_APPRENANT"] == std), "NOTE"] = applied_mark

    return df

def complete_marks(data: list[ExtraMark], df: pd.DataFrame) -> pd.DataFrame:
    for other in data:
        for std in other.marks:
            std_code: int = std[0]
            std_former_mark: float = std[1]
            repeat_exam: bool = std[2]
            note: pd.Series[float] = df.loc[np.logical_and(df["LIBELLE_CONTROLE"] == other.name, df["CODE_APPRENANT"] == std_code), "NOTE"]

            # a student may not be in the exam list if already had a mark
            if len(note) == 0:
                print('1:', other.name, get_std_name_from_id(std_code), std_code, 'new mark (without a previous one)', std_former_mark)
                df = pd.concat([df, pd.DataFrame([["","", other.name, std_code, "", "", std_former_mark]], columns=df.columns)])
            else:
                new_mark: float = note.iloc[0]
                applied_mark: float = 0.0

                if pd.isna(new_mark):
                    applied_mark = std_former_mark
                    print('1:', other.name, get_std_name_from_id(std_code), std_code, 'previous mark (without a new one (NaN))', std_former_mark)
                else:
                    if repeat_exam:
                        applied_mark = compute_new_mark(std_former_mark, new_mark, True)
                        print('1:', other.name, get_std_name_from_id(std_code), std_code, 'new mark', new_mark, 'with a former one', std_former_mark, 'considered as redoublement. Final mark', applied_mark)
                    else:
                        # So 'rattrapage, the new mark is in fact the one of the first exam, and std_former_mark is the new mark from the 'new try'
                        applied_mark = compute_new_mark(new_mark, std_former_mark, False)
                        print('1:', other.name, get_std_name_from_id(std_code), std_code, 'new mark', std_former_mark, 'with a former one', new_mark, 'considered as rattrapage. Final mark', applied_mark)

                df.loc[np.logical_and(df["LIBELLE_CONTROLE"] == other.name, df["CODE_APPRENANT"] == std_code), "NOTE"] = applied_mark
    return df

# complete_marks2 reads from the CSV of the previous year
df_exams_semester1 = complete_marks2(stds_with_specificities_ids, df_exams_semester1, df_exams_semester1_prev)
df_exams_semester2 = complete_marks2(stds_with_specificities_ids, df_exams_semester2, df_exams_semester2_prev)

# complete_marks completes the mark with hard-coded marks
df_exams_semester1 = complete_marks(other_marks_semester1, df_exams_semester1)
df_exams_semester2 = complete_marks(other_marks_semester2, df_exams_semester2)

# display(df_exams_semester1) # type: ignore
# display(df_exams_semester1[df_exams_semester1["CODE_APPRENANT"] == 8234]) # type: ignore



## Boxplots des notes par semestre

In [ ]:
import scipy.stats as stats

def compute_boxplots(dfs: pd.DataFrame, semester: str) -> None:
    bp = dfs.boxplot(column=["NOTE"], by='LIBELLE_CONTROLE', showmeans=True, figsize=(65,10))
    bp.set_ylim(0, 20)
    bp.set_yticks(np.arange(0, 21, 1))
    bp.get_figure().suptitle('')

    for i, data in enumerate(dfs.groupby('LIBELLE_CONTROLE')):
        marks = data[1]['NOTE']
        marks = pd.to_numeric(marks[marks.notnull()], errors='coerce')
        marks = marks[marks.notnull()]
        try:
            z, pval = stats.normaltest(marks)
        except:
            z = 0.0
            pval = 10.0

        plt.text(i + 1.1, 6, "norm: " + str(round(float(pval), 10)))
        plt.text(i + 1.1, 5, "μ: " + str(round(marks.mean(), 3)))
        plt.text(i + 1.1, 4, "m: " + str(round(marks.median(), 3)))
        plt.text(i + 1.1, 3, "σ: " + str(round(marks.std(), 3)))
        plt.text(i + 1.1, 2, "↑: " + str(round(marks.max(), 3)))
        plt.text(i + 1.1, 1, "↓: " + str(round(marks.min(), 3)))

    plt.text(3, 21, "%s - %s - %s" % (year_name, semester, year), weight="bold")
    plt.savefig('graphics/%s_boxplots.pdf' % semester)

    for i, data in enumerate(dfs.groupby('LIBELLE_CONTROLE')):
        marks = data[1]['NOTE']
        marks = marks[marks.notnull()]
        try:
            z, pval = stats.normaltest(marks)
        except:
            z = 0.0
            pval = 10.0

        # if pd.isna(pval):
        #     print(marks)

        if pval < 0.05:
            plt.figure()
            plt.hist(marks.values)
            plt.text(1, 6, data[1]['LIBELLE_CONTROLE'].values[0])
            plt.text(1, 5, str(round(float(pval), 10)))

# compute_boxplots(df_exams_semester1, semester1.name)
# compute_boxplots(df_exams_semester2, semester2.name)


## Table des notes étudiants

### Helpers

In [ ]:
%%HTML

<style>
td,th {
  font-size: 12px
}

tr:hover {background-color: blue !important;}
</style>

### S5

In [ ]:

def apply_style_semester(df: pd.DataFrame, semester: Semester, gap_main_columns: int) -> pdstyle.Styler:
    styler = df.style
    styler.apply(highlight_everyother)
    styler \
        .set_caption("%s %s" % (semester.name, year)) \
        .format(precision=2, decimal=",") \
        .format(precision=0, subset=[semester.get_token_Rank()]) \
        .format(precision=1, decimal=",", subset=[semester.get_token_ECTS(), semester.get_token_ECTS_compensation()]) \
        .set_sticky(axis="index") \
        .apply(lambda x: (x < semester.critical_ects_level).map({True: 'background-color: red; color: black;font-weight: bold;', False: ''}), subset=semester.get_header_table()) \
        .apply(lambda x: (x >= semester.expected_ects).map({True: 'background-color: green; color: black;font-weight: bold;', False: ''}), subset=[semester.get_token_ECTS()]) \
        .apply(lambda x: (x < semester.expected_ects).map({True: 'background-color: orange; color: black;font-weight: bold;', False: ''}), subset=[semester.get_token_ECTS()]) \
        .apply(lambda x: (x < semester.passable_ects_level).map({True: 'background-color: red; color: black;font-weight: bold;', False: ''}), subset=[semester.get_token_ECTS()]) \
        .apply(lambda x: (x >= semester.expected_ects).map({True: 'background-color: green; color: black;font-weight: bold;', False: ''}), subset=[semester.get_token_ECTS_compensation()]) \
        .apply(lambda x: (x < semester.expected_ects).map({True: 'background-color: red; color: black;font-weight: bold;', False: ''}), subset=[semester.get_token_ECTS_compensation()]) \
        .background_gradient(axis=None, subset=[semester.get_token_Rank()], cmap='YlOrRd') \
        .apply(lambda x: ['font-weight: bold;'] * len(x), subset=semester.get_course_names()) \
        .apply(lambda x: ['color: gray;'] * len(x), subset=semester.get_subexam_names()) \
        .apply(lambda x: (pd.isna(x)).map(lambda v: 'background-color: blue; color: white;font-weight: bold;' if v else '')) \
        .format(na_rep='', subset=['Profil']) #\
        # .hide(axis="index")

    for ue in semester.ues:
        styler.apply(highlighterUE, subset=[ue.name], predicate = ue.is_validating_UE(df))

    styler.set_table_styles([{"selector": "td, th", "props": [("border", "1px solid grey !important"), ("padding", ".2em")]}, \
                             {"selector": ".col1, %s" % (semester.get_UE_selectors(gap_main_columns)), "props": [("border-right", "10px solid grey !important")]}])
    return styler

def completeLineStdMarks(marks: pd.DataFrame, nom: str, id: int, semester: Semester) -> str:
    return reduce(lambda x, y: "%s;%s" %(x, y), map(lambda x: str(x),
            [1, semester.get_std_mean(marks, nom, id, specificities), semester.get_std_ECTS_strict(marks, nom, id, specificities),
             semester.get_std_ECTS_compensation(marks, nom, id, specificities)] + semester.get_std_marks(marks, nom, id, specificities)))

def produce_semester_table(semester: Semester, semester_stds: pd.DataFrame, semester_exams: pd.DataFrame, gap_main_columns: int) -> pd.DataFrame:
    marks_semester_base = "ID;Nom;Prénom;%s\n" % (';'.join(semester.get_header_table()))
    marks_semester = marks_semester_base

    for _, row in semester_stds.iterrows():
        std_marks: pd.DataFrame = semester_exams[semester_exams["CODE_APPRENANT"] == row["CODE_APPRENANT"]]
        line = "%s;%s;%s;%s\n" % (row["CODE_APPRENANT"], row["NOM_FAMILLE"], row["PRENOM"], completeLineStdMarks(std_marks, row["NOM_FAMILLE"], row["CODE_APPRENANT"], semester))
        marks_semester += line

    df_marks_semester = pd.read_csv(StringIO(marks_semester), sep= ';', index_col="ID") #, header=[0,1], index_col=1)

    df_marks_semester[semester.get_token_Rank()] = \
        df_marks_semester[[semester.get_token_ECTS(), semester.get_token_ECTS_compensation(), semester.get_token_Avg()]].apply(tuple,axis=1).rank(ascending=False)
    # df_marks_semester[semester.get_token_Rank()] = df_marks_semester['Nom'].rank(ascending=True)

    ## sorting
    # df_marks_semester = df_marks_semester.sort_values(by = "Nom", ascending=True, axis=0)
    df_marks_semester = df_marks_semester.merge(specificities, on=df_marks_semester.index.name, how="left").set_index('ID')
    df_marks_semester = df_marks_semester.merge(ranking_prev_year, on=df_marks_semester.index.name, how="left").set_index('ID')

    df_marks_semester['Profil'] = df_marks_semester['Carac'].astype('string').str.cat(df_marks_semester['RANK'].apply(lambda x: f'{x:.0f}').astype('string'), sep=' - ', na_rep='')
    df_marks_semester.pop("RANK")
    df_marks_semester.pop("Carac")
    df_marks_semester.insert(3, "Profil", df_marks_semester.pop("Profil"))

    df_marks_semester = df_marks_semester.sort_values([semester.get_token_ECTS(), semester.get_token_ECTS_compensation(), semester.get_token_Avg(), 'Nom'], ascending=[False, False, False, True], axis=0)
    # df_marks_semester = df_marks_semester.sort_values("Rank", ascending=True, axis=0)

    # metadata
    metadata_semester = insert_metadata(df_marks_semester, semester.get_header_table())
    styler_metadata= metadata_semester.style.format(precision=2, decimal=",").hide([semester.get_token_Rank()], axis=1)
    display(styler_metadata) # type: ignore
    styler_metadata.to_excel('graphics/stds/%s_means.xlsx' %(semester.name), float_format="%.2f")

    ## Styling
    styler = apply_style_semester(df_marks_semester, semester, gap_main_columns)
    # styler = styler.set_sticky(axis="index").set_sticky(axis="columns") # does not work
    styler.to_excel('graphics/stds/%s.xlsx' %(semester.name), float_format="%.2f")
    styler.to_html('graphics/stds/%s.html' %(semester.name), float_format="%.2f")
    # df_marks_semester.to_csv('graphics/stds/%s.csv' %(semester.name), index = False)

    display(styler)
    return df_marks_semester

df_promo_semester1.reset_index()
df_marks_semester1 = produce_semester_table(semester1, df_promo_semester1, df_exams_semester1, 3)


### S6

In [ ]:
df_promo_semester2.reset_index()
df_marks_semester2 = produce_semester_table(semester2, df_promo_semester2, df_exams_semester2, 3)


### Semester 1 + Semester 2

In [ ]:
def apply_style_semester1_semester2(df: pd.DataFrame) -> pdstyle.Styler:
    styler = df.style
    styler.apply(highlight_everyother)
    styler \
        .set_caption("%s -- %s+%s -- %s" % (year_name, semester1.name, semester2.name, year)) \
        .format(precision=2, decimal=",") \
        .format(precision=1, decimal=",", subset=[semester2.get_token_ECTS(), semester2.get_token_ECTS_compensation(), semester1.get_token_ECTS(), semester1.get_token_ECTS_compensation(), "ECTS", "ECTS+"]) \
        .set_sticky(axis="index") \
        .apply(lambda x: (x >= semester1.expected_ects).map({True: 'background-color: green; color: black;', False: ''}), subset=[semester2.get_token_ECTS_compensation(), semester1.get_token_ECTS_compensation(), semester2.get_token_ECTS(), semester1.get_token_ECTS()]) \
        .apply(lambda x: (x >= (semester1.expected_ects + semester2.expected_ects)).map({True: 'background-color: green; color: black;', False: ''}), subset=["ECTS", "ECTS+"]) \
        .apply(lambda x: (x < (semester1.expected_ects + semester2.expected_ects)).map({True: 'background-color: red; color: black;', False: ''}), subset=["ECTS", "ECTS+"]) \
        .apply(lambda x: (x < semester1.critical_ects_level).map({True: 'background-color: red; color: black;', False: ''}), subset=[semester2.get_token_ECTS(), semester1.get_token_ECTS()]) \
        .apply(lambda x: (x < (semester1.critical_ects_level + semester2.critical_ects_level)).map({True: 'background-color: red; color: black;', False: ''}), subset=["ECTS"]) \
        .apply(lambda x: (x < semester1.expected_ects).map({True: 'background-color: red; color: black;', False: ''}), subset=[semester2.get_token_ECTS(), semester1.get_token_ECTS(), "ECTS+ %s" %(semester1.name), "ECTS+ %s" %(semester2.name)]) \
        .background_gradient(axis=None, subset=["Rank"], cmap='YlOrRd') \
        .format(precision=0, subset=["Rank", semester1.get_token_Rank(), semester2.get_token_Rank(), "Diff Rank"]) \
        .background_gradient(axis=None, subset=["Diff Avg"])

    styler.set_table_styles([{"selector": "td, th", "props": [("border", "1px solid grey !important"), ("padding", ".4em")]}])
    return styler

df_S1_S2 = df_marks_semester1[["Nom", "Prénom", semester1.get_token_Avg(), semester1.get_token_Rank(), semester1.get_token_ECTS(), semester1.get_token_ECTS_compensation()]]\
    .merge(df_marks_semester2[[semester2.get_token_Avg(), semester2.get_token_Rank(), semester2.get_token_ECTS(), semester2.get_token_ECTS_compensation()]], on=df_marks_semester1.index.name, how="left", suffixes=(" %s" % (semester1.name), " %s" %(semester2.name)))

df_S1_S2["ECTS"] = df_marks_semester1[semester1.get_token_ECTS()] + df_marks_semester2[semester2.get_token_ECTS()]
df_S1_S2["ECTS+"] = df_marks_semester1[semester1.get_token_ECTS_compensation()] + df_marks_semester2[semester2.get_token_ECTS_compensation()]
df_S1_S2["Avg"] = (df_marks_semester1[semester1.get_token_Avg()] + df_marks_semester2[semester2.get_token_Avg()]) / 2.0
df_S1_S2["Diff Avg"] = df_S1_S2[semester2.get_token_Avg()] - df_S1_S2[semester1.get_token_Avg()]
df_S1_S2["Diff Rank"] = df_S1_S2["Rank %s" %(semester1.name)] - df_S1_S2["Rank %s" %(semester2.name)]
df_S1_S2["Rank"] = df_S1_S2[["ECTS", "ECTS+", "Avg"]].apply(tuple,axis=1).rank(ascending=False)
# df_S1_S2["Rank"] = df_S1_S2["Avg"].rank(ascending=False)
# df_S1_S2 = df_S1_S2.sort_values("Nom", ascending=True, axis=0)
# df_marks_semester.sort_values([semester.get_token_ECTS(), semester.get_token_ECTS_compensation(), semester.get_token_Avg(), 'Nom'],
df_S1_S2 = df_S1_S2.sort_values(["ECTS", "ECTS+", "Avg"], ascending=False, axis=0)

styled_S1_S2 = apply_style_semester1_semester2(df_S1_S2)
styled_S1_S2.to_excel('graphics/stds/%s_%s.xlsx'%(semester1.name, semester2.name), float_format="%.2f")
styled_S1_S2.to_html('graphics/stds/%s_%s.html'%(semester1.name, semester2.name), float_format="%.2f")
display(styled_S1_S2)


In [ ]:
# Box plots

def compute_boxplots_year(df: pd.DataFrame) -> None:
    bp = df.boxplot(column=['Avg'], figsize=(3,8), showmeans=True)
    bp.set_ylim(0, 20)
    bp.set_yticks(np.arange(0, 21, 1))
    bp.get_figure().suptitle('')

    plt.text(1.1, 5, "μ: " + str(round(df['Avg'].mean(), 3)))
    plt.text(1.1, 4, "m: " + str(round(df['Avg'].median(), 3)))
    plt.text(1.1, 3, "σ: " + str(round(df['Avg'].std(), 3)))
    plt.text(1.1, 2, "↑: " + str(round(df['Avg'].max(), 3)))
    plt.text(1.1, 1, "↓: " + str(round(df['Avg'].min(), 3)))
    plt.figure()

    bp2 = df.boxplot(column=['ECTS', 'ECTS+'], figsize=(3,8), showmeans=True)
    bp2.set_ylim(0, 64)
    bp2.set_yticks(np.arange(0, 65, 5))
    bp2.get_figure().suptitle('')

    plt.text(1.1, 21, "μ: " + str(round(df['ECTS'].mean(), 3)))
    plt.text(1.1, 16, "m: " + str(round(df['ECTS'].median(), 3)))
    plt.text(1.1, 11, "σ: " + str(round(df['ECTS'].std(), 3)))
    plt.text(1.1, 6, "↑: " + str(round(df['ECTS'].max(), 3)))
    plt.text(1.1, 1, "↓: " + str(round(df['ECTS'].min(), 3)))
    plt.text(2.1, 21, "μ: " + str(round(df['ECTS+'].mean(), 3)))
    plt.text(2.1, 16, "m: " + str(round(df['ECTS+'].median(), 3)))
    plt.text(2.1, 11, "σ: " + str(round(df['ECTS+'].std(), 3)))
    plt.text(2.1, 6, "↑: " + str(round(df['ECTS+'].max(), 3)))
    plt.text(2.1, 1, "↓: " + str(round(df['ECTS+'].min(), 3)))
    plt.figure()


compute_boxplots_year(df_S1_S2)

## Jury

### Liste étudiants qui ne valident pas

In [ ]:
from operator import itemgetter

def check_module_ko(value: float, name: str) -> str:
    return "" if pd.isna(value) | (value >= 10) else name

def get_list_modules_ko(ko_semester: pd.DataFrame, modules: list[str]) -> list[str]:
    modulesKO_std: list[str] = []
    for _, row in ko_semester.iterrows():
        modulesKO: list[str] = []
        for mod in modules:
            modulesKO.append(check_module_ko(row[mod], mod))

        modulesKO_std.append(';'.join(filter(lambda name: len(name) > 0, modulesKO)))
    return modulesKO_std


def print_stds_ko(ko_semester: pd.DataFrame, semester: str) -> None:
    print("## %s" % (semester))
    name_col_ko = "KO %s" % (semester)
    display(ko_semester[['Nom', 'Prénom', name_col_ko]])
    print("Nb KO par module du %s pour les étudiants qui ne valident pas le %s:" % (semester, semester))
    print()
    res = ';'.join(map(str, ko_semester[name_col_ko].values.tolist())).split(';')
    print(*sorted(([x,res.count(x)] for x in set(res)), key=itemgetter(1), reverse=True), sep = "\n")


def get_ko_names() -> None:
    ko_S1 = df_marks_semester1[df_marks_semester1["ECTS+ %s" %(semester1.name)] < semester1.expected_ects].copy() #.drop(123456)
    ko_S2 = df_marks_semester2[df_marks_semester2["ECTS+ %s" %(semester2.name)] < semester2.expected_ects].copy() #.drop(123456)
    fct = lambda row: "%s %s" % (row["Prénom"], row["Nom"])

    ko_S1_noms = ko_S1.apply(fct, axis=1)
    ko_S2_noms = ko_S2.apply(fct, axis=1)
    ko_S1_noms = pd.DataFrame({'Nom': ko_S1_noms.values})
    ko_S2_noms = pd.DataFrame({'Nom': ko_S2_noms.values})
    ko_noms = pd.concat([ko_S1_noms, ko_S2_noms]).drop_duplicates()
    ko_both_S1_S2_noms = pd.merge(ko_S1_noms, ko_S2_noms, how ='inner')

    print("## Stats")
    print()
    print("%s étudiants ne valident pas le %s" % (len(ko_S1_noms), semester1.name))
    print("%s étudiants ne valident pas le %s" % (len(ko_S2_noms), semester2.name))
    print("%s étudiants ne valident pas le %s ou le %s" % (len(ko_noms), semester1.name, semester2.name))
    print("%s étudiants ne valident ni le %s ni le %s" % (len(ko_both_S1_S2_noms), semester1.name, semester2.name))
    print()

    ko_S1['KO %s'%(semester1.name)] = get_list_modules_ko(ko_S1, semester1.get_course_names())
    ko_S2['KO %s'%(semester2.name)] = get_list_modules_ko(ko_S2, semester2.get_course_names())
    ko_S1_S2 = ko_S1[['Nom', 'Prénom', "KO %s" %(semester1.name)]].merge(ko_S2[['Nom', 'Prénom', "KO %s"%(semester2.name)]], on=[ko_S1.index.name, "Nom", "Prénom"], how="outer")

    print_stds_ko(ko_S1, semester1.name)
    print()
    print()
    print_stds_ko(ko_S2, semester2.name)
    print()
    print()
    print("## Bilan %s %s" % (semester1.name, semester2.name))
    print("Attention, cette liste résume par semestre. Si un étudiant valide un semestre, ses notes < 10 ne seront pas affichées ici.")
    print()
    display(ko_S1_S2)

get_ko_names()


In [ ]:
# Table
stds_that_dont_pass = df_S1_S2[(df_S1_S2["ECTS+ %s"%(semester1.name)] < semester1.expected_ects) | (df_S1_S2["ECTS+ %s"%(semester2.name)] < semester2.expected_ects)].sort_values("Rank", ascending=True, axis=0)
stds_that_dont_pass_styled = apply_style_semester1_semester2(stds_that_dont_pass)
stds_that_dont_pass_styled.to_excel('graphics/stds/KO_%s.xlsx'%(year_name), float_format="%.2f")
stds_that_dont_pass_styled.to_html('graphics/stds/KO_%s.html'%(year_name), float_format="%.2f")
display(stds_that_dont_pass_styled)


### Production clusters

In [ ]:

def produce_clusters(clusters: list[list[int]]) -> pd.DataFrame:
    df = df_marks_semester1.merge(df_marks_semester2, on=df_marks_semester1.index.name, how="left", suffixes=(" %s"%(semester1.name), " %s"%(semester2.name))) \
        .merge(df_S1_S2[["ECTS", "ECTS+", "Rank", "Diff Rank", "Diff Avg", "Avg"]], on=df_marks_semester1.index.name, how="left")
    for cluster in clusters:
        temp_df = df.loc[cluster].copy().drop(columns=["Nom %s"%(semester2.name), "Prénom %s"%(semester2.name)])
        if isinstance(temp_df, pd.DataFrame):
            styled = apply_style_semester1_semester2(temp_df).hide(axis="index")
            display(styled)
            # temp_df.to_html("graphics/stds/cluster%s.html" % (i), float_format="%.2f")

    return df

# produce_clusters([[123, 456], [789, 456]])


### Par étudiant

In [ ]:
if not os.path.exists(os.path.join('graphics','stds', 'indiv')):
    os.makedirs(os.path.join('graphics','stds', 'indiv'))

def exportByStudent(promo_s1: pd.DataFrame, promo_s2: pd.DataFrame, marks_s1: pd.DataFrame, marks_s2: pd.DataFrame) -> None:
    stds = pd.concat([promo_s1, promo_s2]).drop_duplicates()

    for _, std in stds.iterrows():
        name = "%s-%s" % (std["NOM_FAMILLE"], std["PRENOM"])
        row = [std["CODE_APPRENANT"]]
        s1 = marks_s1.loc[row]
        s2 = marks_s2.loc[row]
        styler_s1 = apply_style_semester(s1, semester1, 3)
        styler_s2 = apply_style_semester(s2, semester2, 3)
        with open("graphics/stds/indiv/%s.html" % (name), 'w') as _file:
            pic = "<img src='%s%s' alt='%s'></img>" %(pictures_folder, name, name)
            _file.write(pic + "\n\n" + styler_s1.to_html(float_format="%.2f") + "\n\n" + styler_s2.to_html(float_format="%.2f"))

exportByStudent(df_promo_semester1, df_promo_semester2, df_marks_semester1, df_marks_semester2)


## Redoublements

In [ ]:
def formatFR(value: float) -> None:
    print("%s;%s" % ('{:.2f}'.format(value).replace('.', ','), 'X' if value <= 11 else ''))

def getValueRedoublant(module: str, notes: dict[str, list[float]], std: int, redoublants: pd.DataFrame) -> None:
    note = redoublants[module][redoublants['ID'] == std].values[0]

    if note > 11:
        # if module not in notes:
        #     notes[module] = []
        drop_stds = notes[module]
        drop_stds += [redoublants["Nom %s"%(semester1.name)][redoublants['ID'] == std].values[0]]

    formatFR(note)



redoublantsID: list[int] = []

# Contrats
def produce_contracts() -> None:
    redoublants = produce_clusters([redoublantsID])

    drop_modules_redoublants: dict[str, list[float]] = dict(zip(\
        semester1.get_course_names() + semester2.get_course_names(),
        [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]))
    redoublants = redoublants.reset_index()

    for num in redoublantsID:
        print(*redoublants[["Nom %s"%(semester1.name), "Prénom %s"%(semester1.name)]][redoublants['ID'] == num].values[0])
        for course in semester1.get_course_names():
            getValueRedoublant(course, drop_modules_redoublants, num, redoublants)
        # getValueRedoublant("PROBA", drop_modules_redoublants, num)
        # getValueRedoublant("ADFD", drop_modules_redoublants, num)
        # getValueRedoublant("CLP", drop_modules_redoublants, num)
        # getValueRedoublant("C", drop_modules_redoublants, num)
        # getValueRedoublant("HI", drop_modules_redoublants, num)
        # getValueRedoublant("PL", drop_modules_redoublants, num)
        # getValueRedoublant("FUS", drop_modules_redoublants, num)
        # getValueRedoublant("PF", drop_modules_redoublants, num)
        # getValueRedoublant("ÉP S5", drop_modules_redoublants, num)
        # getValueRedoublant("SDD", drop_modules_redoublants, num)
        # getValueRedoublant("CPOO1", drop_modules_redoublants, num)
        # getValueRedoublant("RISQ", drop_modules_redoublants, num)
        # getValueRedoublant("ANG S5", drop_modules_redoublants, num)
        # getValueRedoublant("PSH", drop_modules_redoublants, num)
        # getValueRedoublant("EPS S5", drop_modules_redoublants, num)
        print()
        print()
        for course in semester2.get_course_names():
            getValueRedoublant(course, drop_modules_redoublants, num, redoublants)
        # getValueRedoublant("Res", drop_modules_redoublants, num)
        # getValueRedoublant("BD", drop_modules_redoublants, num)
        # getValueRedoublant("Web", drop_modules_redoublants, num)
        # getValueRedoublant("GA", drop_modules_redoublants, num)
        # getValueRedoublant("CX", drop_modules_redoublants, num)
        # getValueRedoublant("Pr", drop_modules_redoublants, num)
        # getValueRedoublant("Appr", drop_modules_redoublants, num)
        # getValueRedoublant("ÉP S6", drop_modules_redoublants, num)
        # getValueRedoublant("Conf", drop_modules_redoublants, num)
        # getValueRedoublant("Ouv", drop_modules_redoublants, num)
        # getValueRedoublant("TAL/ PARAL", drop_modules_redoublants, num)
        # getValueRedoublant("SD/ SEC", drop_modules_redoublants, num)
        # getValueRedoublant("IND", drop_modules_redoublants, num)
        # getValueRedoublant("ANG S6", drop_modules_redoublants, num)
        # getValueRedoublant("SIM", drop_modules_redoublants, num)
        # getValueRedoublant("EPS S6", drop_modules_redoublants, num)
        # getValueRedoublant("PPI", drop_modules_redoublants, num)
        print()
        print()

# Formation groupes (redoublants à enlever de groupes)

# TODO

# for key, module in drop_modules_redoublants.items():
#     print("drop_", key.replace(" ", "").replace(".", "").replace("/", "").replace("É", "e"), ' = ["', end='', sep='')
#     print(*module, end='"', sep='", "')
#     print("]")
